## Variación espacio-temporal precipitación total

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

Directorio

In [1]:
import os

print('> Directorio actual: ', os.getcwd())  
os.chdir('../')
print('> Directorio actual: ', os.getcwd()) 

> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics/terepaima
> Directorio actual:  /media/javier/Compartida/doctorado/ssev-analytics


In [2]:
from utils.MONGO import CONEXION
from utils.UTILS import *
from datetime import datetime
import pandas as pd

from tensorflow import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [3]:
import warnings
warnings.filterwarnings('ignore')

Iniciando estudio de resultados

In [4]:
park = 'terepaima'

In [5]:
DIR = f'./{park}/experiments/ann/'
experiments = [DIR + x for x in os.listdir(DIR)]

In [6]:
# Resultados
pd_summary = pd.concat([pd.read_csv(x) for x in experiments if x.find('summary')!=-1 ])
pd_summary.head()

,epocas,activation,n_neurons,capas,training_mse,training_rmse,training_mae,trainig_mape,trainig_r,test_mse,test_rmse,test_mae,test_mape,test_r
0,133,"['tanh', 'sigmoid', 'sigmoid', 'sigmoid', 'lin...","[5, 1]",2,0.885515,0.784137,0.610215,39.755045,0.739287,0.869858,0.756653,0.611173,38.864065,0.733557
0,275,"['tanh', 'sigmoid', 'sigmoid', 'sigmoid', 'lin...","[5, 10, 1]",3,0.662425,0.438807,0.460751,31.215746,0.854104,0.667997,0.446220,0.472695,30.855092,0.842871
0,489,"['tanh', 'sigmoid', 'sigmoid', 'sigmoid', 'lin...","[5, 10, 5, 1]",4,0.676894,0.458186,0.477713,32.415690,0.847661,0.679742,0.462049,0.492277,31.446784,0.837297
0,87,"['tanh', 'sigmoid', 'sigmoid', 'sigmoid', 'lin...","[5, 10, 10, 5, 1]",5,0.736602,0.542582,0.526350,34.596103,0.819600,0.737137,0.543371,0.538898,34.730908,0.808661


Mejor modelo

In [7]:
# best model
pd_best_model = pd_summary[pd_summary.trainig_r == pd_summary.trainig_r.max()]
pd_best_model

,epocas,activation,n_neurons,capas,training_mse,training_rmse,training_mae,trainig_mape,trainig_r,test_mse,test_rmse,test_mae,test_mape,test_r
0,275,"['tanh', 'sigmoid', 'sigmoid', 'sigmoid', 'lin...","[5, 10, 1]",3,0.662425,0.438807,0.460751,31.215746,0.854104,0.667997,0.44622,0.472695,30.855092,0.842871


Cargando datos de precipitación ANN

In [8]:
pd_precipitacion = pd.read_pickle(f'{DIR}predicciones.pkl')
pd_precipitacion.head()

,park,periodo,year,month,id_point,latitud,longitud,type,precipitacion_mm,prediction_precipitacion_mm,elevacion_media,precipitacion_narx,prediction_ann
0,terepaima,1995-01-01,1995,1,1,9.96,-69.38,training,0.230639,0.214150,1001.651794,0.214150,0.378702
1,terepaima,1995-02-01,1995,2,1,9.96,-69.38,training,0.216488,0.200263,1001.651794,0.200263,0.463370
2,terepaima,1995-03-01,1995,3,1,9.96,-69.38,training,2.480125,2.011823,1001.651794,2.011823,1.193321
3,terepaima,1995-04-01,1995,4,1,9.96,-69.38,training,2.167091,2.523585,1001.651794,2.523585,2.667052
4,terepaima,1995-05-01,1995,5,1,9.96,-69.38,training,2.611307,2.906471,1001.651794,2.906471,3.381478


Validación por puntos

In [9]:
list_test = []

for id in pd_precipitacion.sort_values('id_point').id_point.unique():

    data = pd_precipitacion.sort_values('periodo').query(f"id_point=={id}")

    test = metrics(observado=data['precipitacion_narx'],
                prediccion=data['prediction_ann'],)
    test['id_point'] = id

    list_test.append( pd.DataFrame(test,index=[0]) )

pd_test_id = pd.concat(list_test)
pd_test_id.to_csv(f'./{park}/data/ann_precipitacion_metricas.csv',index=False)
pd_test_id

,mape,mae,mse,rmse,r2,id_point
0,46.408772,0.658908,0.885758,0.784567,0.450286,1
0,33.600237,0.327168,0.419203,0.175731,0.823004,2
0,23.750456,0.145936,0.191409,0.036637,0.966229,3
0,32.460927,0.437459,0.569309,0.324113,0.839936,4
0,23.816724,0.279847,0.395204,0.156186,0.908247,5
0,17.170160,0.268975,0.372368,0.138658,0.933817,6
0,23.717627,0.775248,0.984836,0.969902,0.832394,7
0,33.445550,0.662335,0.845052,0.714113,0.799019,8
0,45.921871,0.612386,0.814066,0.662704,0.814487,9


Gráfico

In [10]:
rows=3
cols=3

column_titles = list(map(lambda x: 'Punto id:' + str(int(x)), pd_precipitacion.sort_values('id_point',ascending=True).id_point.unique().tolist() ))

fig = make_subplots(rows=rows,
                    cols=cols,
                    subplot_titles=column_titles,
                    horizontal_spacing=0.1,
                    vertical_spacing=0.13)

row_order = pd_precipitacion.sort_values('id_point',ascending=True).id_point.unique().reshape(rows,cols).tolist()

marker_line_width=1.5
marker_size=1.5
line_width=2

for row in range(len(row_order)):

    list_row = row_order[row]
    for col in range(len(list_row)):
        
        if row_order[row][col]==1:
            showlegend=True
        else:
            showlegend=False
            
            
        data_fig = pd_precipitacion.sort_values('id_point',ascending=True).query(f"id_point=={row_order[row][col]}")
        data_fig = data_fig.sort_values('periodo',ascending=True)

        fig.add_trace(
            go.Scatter(x=data_fig['periodo'],
                       #y=data_fig.total_precipitation_mm,
                       y=data_fig['precipitacion_narx'],
                       mode='lines+markers',
                       name='p(t)',# Id: {id}'.format(id=id_),
                       marker_symbol='square',
                       marker_line_width=marker_line_width,
                       marker_size=marker_size,
                       marker_line_color='#4F64B4',
                       marker_color='#4F64B4', 
                       line=dict(color='#004CFF', width=line_width),
                       legendgroup='group1',
                       showlegend=showlegend,
                       
                      ),
            row=row+1, 
            col=col+1,
                
        )
        
        fig.add_trace(
            go.Scatter(x=data_fig['periodo'],
                       #y=data_fig.total_precipitation_mm,
                       y=data_fig['prediction_ann'],
                       mode='lines+markers',
                       name='Predicción',# Id: {id}'.format(id=id_),
                       #line=dict(color='#00B4B1', width=2),
                       marker_symbol='star',
                       marker_line_width=marker_line_width,
                       marker_size=marker_size,
                       marker_line_color='#C60000',
                       marker_color='#C60000',
                       line=dict(color='#00C6BB', width=line_width,dash='dot'),
                       legendgroup='group1',
                       showlegend=showlegend,
                       
                      ),
            row=row+1, 
            col=col+1,
                
        )

        
        fig.update_xaxes(title_text='Mes',title_font=dict(size=12),
                         range=[data_fig.periodo.max() - pd.DateOffset(months=4*12),
                                data_fig.periodo.max()])
        fig.update_yaxes(title_text='Precipitación (mm)',
                         title_font=dict(size=12),
                         range=[0,10]
                        )

# fig['layout']['title']['y']=0.98
# fig['layout']['margin']['t']=100
        
#        
fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)        
fig.update_layout( height=800,
                   width=1500,
                   showlegend = True,
                   font = dict(size = 14),
                   template='plotly_white',
                   title_text=f"""
                             Modelo Espacio Temporal: Precipitación total periodo {pd_precipitacion.periodo.min().strftime('%Y/%m')} al {pd_precipitacion.periodo.max().strftime('%Y/%m')}
                             <br><sup>Parque {'Terepaima'}</sup>
                             """,
                   #xaxis_range=['1980-01-01','2023-01-01'],
                   legend_title_text='Serie',
                   legend_title = dict( font = dict(size = 14)),
                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   legend = dict(
                                #orientation="h",
                                yanchor="bottom",
                                y=1.05,
                                xanchor="right",
                                x=1,
                                font_size=14
                            )
                 )


fig.write_image(f'./{park}/figures/ann_precipitacion_terepaima.png')
fig.show()

# Preparando datos NDVI

### Conexión Mongo DB

In [11]:
# Creando la conexión con MongoDB
db = CONEXION.conexion()
db.list_collection_names()

['estimateMeteorological', 'meteorological', 'estimateSSTNino34', 'SSTNino34']

Descargando información

In [12]:
# Realizando consulta
meteorological = db.meteorological.find({"park":park})

# Generando pandas dataframe
pd_ndvi = pd.DataFrame([file for file in meteorological])
pd_ndvi['periodo'] = pd_ndvi.time.apply(lambda x: datetime.fromordinal(x))
pd_ndvi['mes_year'] =  pd_ndvi['periodo'].dt.strftime('%B-%Y')
#data_pandas.index = pd.to_datetime(data_pandas.periodo)

pd_ndvi.head()

,_id,id_point,park,time,elevacion_maxima,elevacion_media,elevacion_mediana,latitud,longitud,ndvi_maxima,ndvi_media,ndvi_mediana,precipitacion_mm,time_actualizacion,periodo,mes_year
0,63548350343c9f2921dbb11e,1,terepaima,719163,1632.0,1001.651794,986.0,9.96,-69.38,NaN,NaN,NaN,1.698088,738450,1970-01-01,January-1970
1,63548350343c9f2921dbb12a,1,terepaima,719194,1632.0,1001.651794,986.0,9.96,-69.38,NaN,NaN,NaN,0.460838,738450,1970-02-01,February-1970
2,63548350343c9f2921dbb147,1,terepaima,719222,1632.0,1001.651794,986.0,9.96,-69.38,NaN,NaN,NaN,0.559042,738450,1970-03-01,March-1970
3,63548350343c9f2921dbb165,1,terepaima,719253,1632.0,1001.651794,986.0,9.96,-69.38,NaN,NaN,NaN,1.815903,738450,1970-04-01,April-1970
4,63548350343c9f2921dbb174,1,terepaima,719283,1632.0,1001.651794,986.0,9.96,-69.38,NaN,NaN,NaN,3.538272,738450,1970-05-01,May-1970


Gráficos

In [13]:
rows=3
cols=3

column_titles = list(map(lambda x: 'Punto id:' + str(int(x)), pd_ndvi.sort_values('id_point',ascending=True).id_point.unique().tolist() ))

fig = make_subplots(rows=rows,
                    cols=cols,
                    subplot_titles=column_titles,
                    horizontal_spacing=0.1,
                    vertical_spacing=0.13)


row_order = pd_ndvi.sort_values('id_point',ascending=True).id_point.unique().reshape(rows,cols).tolist()
    
for row in range(len(row_order)):

    list_row = row_order[row]
    for col in range(len(list_row)):
        
        if row_order[row][col]==1:
            showlegend=True
        else:
            showlegend=False

        data_fig = pd_ndvi.query(f"id_point=={row_order[row][col]}")
        data_fig = data_fig.sort_values('periodo',ascending=True)

        fig.add_trace(
            go.Scatter(x=data_fig['periodo'],
                        y=data_fig['ndvi_media'],
                        mode='lines',
                        name='NDVI',# Id: {id}'.format(id=id_),
                        line=dict(color='#008302', width=2),
                        legendgroup='group1',
                        showlegend=showlegend,
                        
                        ),
            row=row+1, 
            col=col+1
                
        )
        
        fig.update_xaxes(title_text='Mes',
                            title_font=dict(size=12),
                        range=[ data_fig.dropna().periodo.min(),
                                data_fig.periodo.max()])
        fig.update_yaxes(title_text='NDVI',
                            title_font=dict(size=12),
                            range=[0,1]
                        )
        
fig['layout']['title']['y']=0.98
fig['layout']['margin']['t']=100
        
#        
fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)        
fig.update_layout( height=800,
                   width=1500,
                   showlegend = True,
                   font = dict(size = 14),
                   template='plotly_white',
                   title_text=f"""
                             NDVI periodo {pd_ndvi.dropna().periodo.min().strftime("%Y/%m")} al {pd_ndvi.dropna().periodo.max().strftime("%Y/%m")}
                             <br><sup>Parque {'Terepaima'}</sup>
                             """,
                #    xaxis_range=[data_fig.periodo.max() - pd.DateOffset(months=4*12),
                #                 data_fig.periodo.max()],
                  legend_title_text='Serie',
                  legend_title = dict( font = dict(size = 14)),
                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   legend = dict(
                                #orientation="h",
                                yanchor="bottom",
                                y=1.05,
                                xanchor="right",
                                x=1,
                                font_size=14
                            )
                 )

fig.write_image(f'./{park}/figures/ndvi_{park}.png')

fig.show()

In [14]:
pd_model = pd.merge(pd_precipitacion,pd_ndvi[[ 'id_point', 'latitud','longitud', 'ndvi_media','periodo']],
                    on=['periodo','id_point','latitud','longitud'],
                    how='left')

pd_model.to_pickle(f'./{park}/data/ann_precipitacion_terepaima.pkl')